<a href="https://colab.research.google.com/github/abdelaziz2003vvb/DCGAN---Deep-Convolutional-Generative-Adversarial-Networks/blob/main/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Article : Unsupervised representation learning with deep convolutional generative adversarial networks

100z->  project and reshape (4X4-1024) -> conv1 (8X8-512) -> conv2 (16x16-256) ->conv3 (32X32-128) ->conv4 -> G(z)

so discriminator looks the same as the generator but the Opposite

#Architecture guidelines for stable Deep convolutionnel GANs


*   Replace any pooling layers with strided convolutions (discriminator) and fractional-strided convolutions (generator).

*  Use batchnorm in both the generator and the discriminator.
*   Remove fully connected hidden layers for deeper architectures.

*  Use ReLU activation in generator for all layers except for the output, which uses Tanh.

*   Use LeakyReLU activation in the discriminator for all layers.





In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [ ]:
from torch.nn.modules.activation import LeakyReLU
class Discriminator(nn.Module):
  def __init__(self,channels_img,features_d):
    super(Discriminator,self).__init__()
    self.disc=nn.Sequential(
        #Input : N x channnels_img x 64 x 64
        nn.Conv2d(
            channels_img,features_d,kernel_size=4,stride=2, padding=1
        ),#32x32
        nn.LeakyReLU(0.2),
        self._block(features_d,features_d*2,4,2,1),#16x16
        self._block(features_d*2,features_d*4,4,2,1),#8x8
        self._block(features_d*4,features_d*8,4,2,1),#4x4
        nn.Conv2d(features_d*8,1,kernel_size=4,stride=2,padding=0),#1x1
        nn.Sigmoid(),

    )

  def _block(self,in_channels, out_channels, kernel_size, stride , padding):
    return nn.Sequential(
        nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias=False,
        ),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2),
    )

  def forward(self,x):
    return self.disc(x)

In [ ]:
from torch.nn.modules.activation import LeakyReLU
class Generator(nn.Module):
  def __init__(self,z_dim,channels_img,features_g):
    super(Generator,self).__init__()
    self.net=nn.Sequential(
        # Input : N x z_dim x 1 x 1
        self._block(z_dim,features_g*16,4,1,0),#N x f_g *16 x 4 x 4
        self._block(features_g*16,features_g*8,4,2,1),#8x8
        self._block(features_g*8,features_g*4,4,2,1),#16 X 16
        self._block(features_g*4,features_g*2,4,2,1),#32 X 32
        nn.ConvTranspose2d(features_g*2,channels_img ,kernel_size=4,stride=2,padding=1),
        nn.Tanh(),# [-1 , 1]

    )

  def _block(self,in_channels, out_channels, kernel_size, stride , padding):
    return nn.Sequential(
        nn.ConvTranspose2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias=False,
        ),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
    )

  def forward(self,x):
    return self.net(x)

In [ ]:
def initialize_weights(model):
  for m in model.modules():
    if isinstance(m,nn.Conv2d):
      nn.init.normal_(m.weight.data,0.0,0.02)

    if isinstance(m,nn.ConvTranspose2d):
      nn.init.normal_(m.weight.data,0.0,0.02)

    if isinstance(m,nn.BatchNorm2d):
      nn.init.normal_(m.weight.data,0.0,0.02)


In [ ]:
def test():
  N, in_channels , H , W=8,3,64,64
  z_dim=100
  x=torch.randn((N,in_channels,H ,W))
  disc = Discriminator(in_channels,8)
  initialize_weights(disc)
  assert disc(x).shape == (N,1,1,1)
  gen =Generator(z_dim,in_channels,8)
  initialize_weights(gen)
  z=torch.randn((N,z_dim,1 ,1))
  assert gen(z).shape ==(N,in_channels, H ,W)
  print("Success")


In [ ]:
test()

Success


In [ ]:
 #  steup hyperparametres
device = "cuda" if torch.cuda.is_available() else "cpu"
lr=2e-4
batch_size=128
image_size=64
channel_img=1
z_dim=100
num_epochs=5
features_disc=64
features_gen=64
transform = torchvision.transforms.Compose([
    transforms.Resize(image_size),
    torchvision.transforms.ToTensor(),
    transforms.Normalize(
        [0.5 for _ in range(channel_img)], [0.5 for _ in range(channel_img)]),   # IMPORTANT for Tanh generator
])


dataset = datasets.MNIST(root="dataset/",train =True , transform=transform, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.31MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 126kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.23MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.54MB/s]


In [ ]:
disc = Discriminator(channel_img,features_disc).to(device)
gen = Generator(z_dim, channel_img, features_gen).to(device)

In [ ]:
initialize_weights(gen)
initialize_weights(disc)

In [ ]:
opt_disc = optim.Adam(disc.parameters(), lr=lr,betas=(0.5,0.999))
opt_gen = optim.Adam(gen.parameters(), lr=lr, betas=(0.5,0.999))

In [ ]:
criterion = nn.BCELoss()
fixed_noise = torch.randn(32,z_dim, 1, 1).to(device)
writer_real = SummaryWriter(f"logs/real")
writer_fake = SummaryWriter(f"logs/fake")
step = 0

gen.train()
disc.train()

for epoch in range(num_epochs):
    # Target labels not needed! <3 unsupervised
    for batch_idx, (real, _) in enumerate(loader):
        real = real.to(device)
        noise = torch.randn(batch_size,z_dim, 1, 1).to(device)
        fake = gen(noise)

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
        disc_real = disc(real).reshape(-1)
        loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake.detach()).reshape(-1)
        loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        loss_disc = (loss_disc_real + loss_disc_fake) / 2
        disc.zero_grad()
        loss_disc.backward()
        opt_disc.step()

        ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        output = disc(fake).reshape(-1)
        loss_gen = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

        # Print losses occasionally and print to tensorboard
        if batch_idx % 100 == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                  Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise)
                # take out (up to) 32 examples
                img_grid_real = torchvision.utils.make_grid(real[:32], normalize=True)
                img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)

                writer_real.add_image("Real", img_grid_real, global_step=step)
                writer_fake.add_image("Fake", img_grid_fake, global_step=step)

            step += 1

